In [ ]:
import os

import pandas as pd
import sqlalchemy as sal

## Creazione de Postgre DB tramite Docker:
Le frasi in grassetto si intendono stringhe di comando da eseguire nella CLI. \
1. Pulliamo da DockerHub un'immagine Postgres
2. Runniamo il container
**docker run --name DBtutorial -e POSTGRES_PASSWORD=xxxx -p 5432:5432 -d postgress**
il comando *docker run* servirebbe per avviare un container su un'immagine già montata e presente nel filesystem ma dal momento che questa non è già presente in locale, eseguire comunque il comando *docker run*, con gli argomenti indicati, attiverà automaticamente la pull e la build dell'immagine con relativo avvio del container.

Con il comando **docker ps** verifichiamo che il nostro container "DBtutorial" sia attivo 

A questo punto interagiamo con il container per creare il DB posgress all'interno di esso: \

**docker exec -it DBtutorial bash** per entrare nel container ed interagire tramite la shell bash

**psql** per connettersi al server (ci risponderà con un errore di permessi "failed: FATAL:  role "root" does not exist" \
**psql -U postgres** Per ottenere i permessi root usando l'utente postreges \

**\l** per visualizzare la lista dei DB già presenti \

Creiamo il nostro database "pokemondb":\
**CREATE DATABASE "pokemondb";** NB: il ";" in chiusura del comando

Proseguiamo con la creazione delle tabelle utilizzando il ORM di SQLAlchemy:

In [ ]:
from sqlalchemy import create_engine

username = os.getenv('DB_USER', '')
password = os.getenv('DB_PASS', '')
host = os.getenv('DB_HOST', '')
port = os.getenv('DB_PORT', '')

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/pokemondb", echo=True)

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

In [ ]:
Base = declarative_base()

In [ ]:
from sqlalchemy import Column, String, DateTime, Integer, CHAR, ForeignKey

In [ ]:
class Pokemon(Base): #eredita dalla classe Base
    __tablename__ = "Pokemon" #definiamo il nome tabella con il dundermethod specifico
    
    __table_args__ = {'extend_existing': True}
    
    id_pkm = Column(
        Integer(), 
        primary_key=True #permette di assegnare la caratteristica di primary-key
    )
    nome = Column(
        String(25), #possiamo chiamare la classe del datatype per assegnare il valore massimo dell'attributo
        nullable=False, #fa sì che l'attributo non possa essere nullo
    ) #fa sì che l'attributo sia univoco
    
    tipo = Column(
        String(25), #possiamo chiamare la classe del datatype per assegnare il valore massimo dell'attributo
        nullable=False,
        unique=True) #fa sì che l'attributo sia univoco
    
    generazione = Column(
        Integer, 
        nullable=False)
    
    bst = Column(
        Integer,
        nullable=False)

    
    def __init__(self, id_pkm, nome, tipo, generazione, bst):
        self.id = id_pkm
        self.nome = nome
        self.tipo = tipo
        self.generazione = generazione
        self.bst = bst
        
    
    def __repr__(self): #definiamo come vogliamo rappresentare l'oggetto della classe quando viene visualizzato
        return f"({self.id}) {self.nome}, {self.tipo}, {self.generazione}, base_stats_total: {self.bst})"

In [ ]:
engine.dialect

In [ ]:
Base.metadata.create_all(bind=engine)

In [ ]:
Pokemon.__table__

In [ ]:
class Debolezze(Base):
    __tablename__ = "Debolezze"
    
    __table_args__ = {'extend_existing': True}
    
    d_id = Column("d_id", Integer, primary_key=True)
    tipo = Column("tipo", String, ForeignKey("Pokemon.tipo"), unique=True)
    debolezza = Column("debolezza", String)
                  
    def __init__(self, d_id, tipo, debolezza):
        self.id = d_id
        self.tipo = tipo
        self.debolezza = debolezza
    
    def __rpr__(self):
        return f"({self.id}, il tipo {self.tipo} è debole al tipo {self.debolezza})"

In [ ]:
Base.metadata.create_all(bind=engine)

In [ ]:
Debolezze.__table__

In [ ]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
pkm_1 = Pokemon(1, "Bulbasaur", "Erba", 1, 318)

In [ ]:
pkm_2 = Pokemon(2, "Charmander", "Fuoco", 1, 309)

In [ ]:
pkm_3 = Pokemon(3, "Squirtle", "Acqua", 1, 314)

In [ ]:
with engine.connect() as con:
    statement = text('INSERT INTO "Pokemon" (nome, tipo, generazione, bst) VALUES ({}, {}, 1, 421)'.format("'Gengar'", "'Spettro'"))  
    con.execute(statement)

In [ ]:
session.add_all([pkm_1, pkm_2, pkm_3])

In [ ]:
session.commit()

In [ ]:
deb_1 = Debolezze(1, pkm_1.tipo, "Fuoco")
deb_2 = Debolezze(2, pkm_2.tipo, "Acqua")
deb_3 = Debolezze(3, pkm_3.tipo, "Erba")

In [ ]:
session.add_all([deb_1, deb_2, deb_3])

In [ ]:
session.commit()

In [ ]:
with engine.connect() as con:

    rs = con.execute('SELECT * FROM "Pokemon"')

    for row in rs:
        print(row)

In [ ]:
with engine.connect() as con:

    rs = con.execute('SELECT * FROM "Debolezze"')

    for row in rs:
        print(row)

In [ ]:
from sqlalchemy import text

In [ ]:
with engine.connect() as con:
    query = text('SELECT bst, nome FROM "Pokemon" WHERE nome LIKE {}'.format("'%saur'"))
    rs = con.execute(query)

    for row in rs:
        print(row)